# 🚀 Tecton Quickstart
---

Want to see what Tecton can do? In this 10-minute quickstart, you'll build a streaming feature pipeline that can process real-time transaction data and serve features with millisecond latency. Perfect for fraud detection, real-time recommendations, or any application that needs fresh feature values.

### What You'll Build

We'll create a simple but powerful feature pipeline that:

- Ingests streaming transaction data in real-time
- Computes running totals over different time windows
- Serves feature values with sub-second latency

### What You'll Learn

- How to define features in Tecton using Python
- How to test features directly in your notebook
- How to ingest real-time data and see immediate updates

No prior Tecton experience needed - if you're comfortable with Python, you can complete this tutorial. Ready to get started?

##### 💡 **NOT YET A TECTON USER?**

Sign-up at [explore.tecton.ai](https://explore.tecton.ai) for a free account that lets you try out this tutorial and explore Tecton's Web UI.

---

Tecton helps you build and productionize AI applications by making it easy to define, test, and deploy features for model training and serving.

Let's take a quick look at how you can build a low-latency streaming feature for a fraud detection use case using nothing but Python.

For an end-to-end tutorial that takes you from raw data to a real-time model, check out [Building a Production AI Application with Tecton](https://docs.tecton.ai/docs/tutorials/building-a-production-ai-application).

## ⚙️ Install Tecton

First things first, let's install the Tecton SDK and other libraries used in this tutorial by running the cell below.

**NOTE**: When the `pip install` is finished you will need to restart the session in order to continue. Click "Restart session" in the dialog box or in the notebook cell.

In [ ]:
!pip install 'tecton[rift]==1.1.0' gcsfs s3fs --force

## ✅ Log in to Tecton

Make sure to hit `enter` after pasting in your authentication token.

In [ ]:
import tecton

tecton.login("https://explore.tecton.ai")

That's it for setup! Your environment is ready for building features. Let's create your first streaming feature pipeline.

## 🧪 Define and test a streaming feature

Using Tecton's Feature Engineering Framework we will define 3 new features for our fraud detection model:

- A user's total transaction amount in the last 1 minute
- A user's total transaction amount in the last 1 hour
- A user's total transaction amount in the last 30 days

To do so, we will first define a [Stream Source](https://docs.tecton.ai/docs/defining-features/data-sources/creating-a-data-source/creating-and-testing-a-push-source) which tells Tecton where to retrieve events online and offline. For the online path, we've configured Tecton to accept real-time events via an HTTP ingestion API which we will try sending records to in the next step. For the offline path, we've pointed Tecton to an S3 path that contains a historical record of our stream. Tecton uses this path for offline development and backfills.

Next we define a [Stream Feature View](https://docs.tecton.ai/docs/defining-features/feature-views/stream-feature-view/stream-feature-view-with-rift) which can create one or more features via transformations against the Stream Source. In a Stream Feature View, we can run Python transformations on incoming records, and optionally apply time-windowed [aggregations](https://docs.tecton.ai/docs/defining-features/feature-types/aggregation-engine) via the `aggregations` parameter. These transformations run identically online and offline which is critical for preventing skew.

One of the great things about Tecton is that we can test our features right away using historical data via the `get_features_in_range` method. This helps us validate that everything is working as expected before we start processing real-time data:

In [ ]:
from tecton import *
from tecton.types import *
from datetime import datetime, timedelta


transactions_stream = StreamSource(
    name="transactions_stream",
    stream_config=PushConfig(),
    batch_config=FileConfig(
        uri="s3://tecton.ai.public/tutorials/transactions.pq",
        file_format="parquet",
        timestamp_field="timestamp",
    ),
    schema=[Field("user_id", String), Field("timestamp", Timestamp), Field("amount", Float64)],
)


user = Entity(name="user", join_keys=[Field("user_id", String)])


@stream_feature_view(
    source=transactions_stream,
    entities=[user],
    mode="pandas",
    timestamp_field="timestamp",
    features=[
        Aggregate(input_column=Field("amount", Float64), function="sum", time_window=timedelta(minutes=1)),
        Aggregate(input_column=Field("amount", Float64), function="sum", time_window=timedelta(hours=1)),
        Aggregate(input_column=Field("amount", Float64), function="sum", time_window=timedelta(days=30)),
    ],
)
def user_transaction_amount_totals(transactions_stream):
    return transactions_stream[["user_id", "timestamp", "amount"]]


# Test the feature locally using historical data
df = (
    user_transaction_amount_totals.get_features_in_range(start_time=datetime(2022, 1, 1), end_time=datetime(2022, 2, 1))
    .to_pandas()
    .fillna(0)
)
df.head(5)

The `_valid_to` and `valid_from` columns specify the time range for which the
row of feature values is valid. See
[Offline Retrieval Methods](../reading-feature-data/reading-feature-data-for-training/offline-retrieval-methods)
for more information.

These results show us that our feature is working! For each user, we can see
their transaction totals over different time windows. The different windows let
us capture both immediate activity (1-minute window), recent patterns (1-hour
window), and longer-term behavior (30-day window).

In the next section, we'll make this real-time by sending in live data and
watching these features update instantly!

## ⚡️ Ingest data and retrieve updated feature values in real-time

Tecton objects get registered via a [declarative workflow](https://docs.tecton.ai/docs/the-feature-development-workflow#5-copy-definitions-into-your-teams-feature-repository). Features are defined as code in a repo and applied to a workspace (like a project) in a Tecton account using the Tecton CLI. This declarative workflow enables productionization best practices such as "features as code," CI/CD, and unit testing.

We've gone ahead and [registered these features](https://explore.tecton.ai/app/repo/prod/features/user_transaction_amount_totals/pipeline) so you can try ingesting new events and querying for online features. After features get registered with a workspace, Tecton handles the [backfilling and ongoing materialization](https://explore.tecton.ai/app/repo/prod/features/user_transaction_amount_totals/materialization) of data to the offline and online store for training and serving.

**This step requires generating an API key, which you can do [here](https://explore.tecton.ai/app/settings/accounts-and-access/service-accounts?create-service-account=true). Copy the generated key and paste it in below.**

In [ ]:
import random, string

tecton.login(tecton_url="https://explore.tecton.ai", tecton_api_key="your-api-key") # replace with your API key

# Fetch registered Tecton objects
ws = tecton.get_workspace("prod")
ds = ws.get_data_source("transactions_stream")
fv = ws.get_feature_view("user_transaction_amount_totals")

# Generate a random user_id for the next step
user_id = "user_" + "".join(random.choices(string.digits, k=7))
print("Generated random user id: " + user_id)

### 🔥 Run this repeatedly and watch the features update!

**Note: It may take a few minutes for your API key permissions to update.**

In [ ]:
# Ingest a new transaction with any amount you want
record = ds.ingest({
    "user_id": user_id,
    "timestamp": datetime.utcnow(),
    "amount": 100
})
print(f"Ingested record for '{user_id}':")
print(record)

# Read updated features via Tecton's HTTP API
features = fv.get_online_features(join_keys={"user_id": user_id}).to_dict()
print(f"\nUpdated features for '{user_id}':")
print(features)

Here's where it gets interesting - try running the ingest code multiple times
with different amounts! You'll see how the different time windows behave:

- The 1-minute window shows the most recent activity - perfect for detecting sudden bursts of transactions
- The 1-hour window accumulates transactions over a longer period - useful for identifying short-term patterns
- The 30-day window maintains a longer-term view - great for understanding typical user behavior

This kind of real-time feature computation is incredibly powerful for:

- Fraud detection systems that need to spot unusual spending patterns immediately
- Recommendation engines that adapt to user behavior in real-time
- Any application where fresh feature values make a difference

Under the hood, Tecton is handling all the complexity of:

- Streaming data ingestion
- Real-time feature computation
- Time window management
- Low-latency feature serving

In a real application, you'd send this data directly to Tecton's HTTP API for the best performance. The `.ingest()` method we're using here is great for testing and development, but for production use cases, you'll want to use the HTTP API directly.

Let's wrap up and look at what you've accomplished!

## ⭐️ Conclusion

Congratulations! You've just built a real-time feature pipeline that can:

- Ingest streaming transaction data
- Calculate features over multiple time windows
- Serve feature values with sub-second latency

### What We Built

- A streaming data source that accepts real-time events
- A feature view that computes running totals over three time windows:
  - 1 minute (very recent activity)
  - 1 hour (recent patterns)
  - 30 days (long-term patterns)
- A real-time serving endpoint for these features

### Key Concepts You've Learned

- How to define a `StreamSource` for real-time data
- How to create time-windowed aggregation features
- How to test features with historical data
- How to send and receive real-time updates

### Next Steps

1. **Explore More Features**

   - Try different aggregation functions (mean, count, max)
   - Add more time windows
   - Combine multiple features together

2. **Ready for Production?** Try our
   ["Building a Production AI Application"](https://docs.tecton.ai/docs/tutorials/building-a-production-ai-application)
   tutorial to learn:
   - How to set up monitoring
   - How to generate training data
   - Best practices for production deployment

[Try Tecton for free](https://www.tecton.ai/explore/).